In [3]:
%matplotlib inline

import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')

In [11]:
## handle bottom-up data
#csvs1 = glob.glob('./bottomup/*csv')
#data1 = {}
#for f in csvs1:
#    content = open(f).read().splitlines()[1:]
#    for line in content:
#        tmp = line.split(',', 1)[0]
#        if tmp not in data1:
#            data1[tmp] = []
#        data1[tmp].append('%s,%s' % (os.path.basename(f).replace('.csv', ''), line))
#for k, v in data1.iteritems():
#    with open('./stock_data/bu_%s.csv' % k, 'wb') as fout:
#        fout.write('\n'.join(v))

# handle top-down data
#csvs2 = glob.glob('./topdown/*csv')
#data2 = {}
#for f in csvs2:
#    content = open(f).read().splitlines()[1:]
#    for line in content:
#        d, v = line.split(',', 1)
#        if d not in data2:
#            data2[d] = []
#        data2[d].append(v)
#with open('./stock_data/td.csv', 'wb') as fout:
#    for k, v in data2.iteritems():
#        fout.write('%s,%s\n' % (k, ','.join(v)))

In [19]:
t = np.loadtxt('./stocks_data/bu_00164V10.csv', delimiter=',', dtype=str)
t[:, 2:-1] = t[:, 2:-1].astype(np.float64)

array([['20110731', '00164V10', '0.0349897907541', ..., '0.0', '0.0',
        '2011-07-31'],
       ['20110831', '00164V10', '-0.0149891992693', ..., '0.0', '0.0',
        '2011-08-31'],
       ['20110930', '00164V10', '-0.152797324281', ..., '0.0', '0.0',
        '2011-09-30'],
       ['20111031', '00164V10', '0.125949179814', ..., '0.0', '0.0',
        '2011-10-31'],
       ['20111130', '00164V10', '0.0286015755692', ..., '0.0', '0.0',
        '2011-11-30']], 
      dtype='|S18')

In [24]:
np.hstack((t[:, 0].reshape(-1, 1), t[:, 2:-1]))[:5]

array([['20110731', '0.0349897907541', '0.0', ..., '0.0', '0.0', '0.0'],
       ['20110831', '-0.0149891992693', '0.0', ..., '0.0', '0.0', '0.0'],
       ['20110930', '-0.152797324281', '0.0', ..., '0.0', '0.0', '0.0'],
       ['20111031', '0.125949179814', '0.0', ..., '0.0', '0.0', '0.0'],
       ['20111130', '0.0286015755692', '0.0', ..., '0.0', '0.0', '0.0']], 
      dtype='|S18')

In [28]:
mat = np.hstack((t[:, 0].reshape(-1, 1), t[:, 2:-1])).astype(np.float64)
mat[:5]

array([[  2.01107310e+07,   3.49897908e-02,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.01108310e+07,  -1.49891993e-02,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.01109300e+07,  -1.52797324e-01,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.01110310e+07,   1.25949180e-01,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.01111300e+07,   2.86015756e-02,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])